# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
# your code here
orders_df = pd.read_csv('Orders.zip')
orders_df.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [3]:
# identify and group customers by their spending.

sum_byInvoice = orders_df.groupby('CustomerID').agg({'amount_spent': 'sum'})
sum_byInvoice

,amount_spent
CustomerID,
12346,77183.60
12347,4310.00
12348,1797.24
12349,1757.55
12350,334.40
...,...
18280,180.60
18281,80.82
18282,178.05


In [4]:
# Select and divide the customers into two categories according to their spending
amount_spentLabels = ['Preferred Customers', 'VIP']
bins = pd.qcut(sum_byInvoice['amount_spent'],2, labels=amount_spentLabels,precision=0)
bins.head(10)

CustomerID
12346                    VIP
12347                    VIP
12348                    VIP
12349                    VIP
12350    Preferred Customers
12352                    VIP
12353    Preferred Customers
12354                    VIP
12355    Preferred Customers
12356                    VIP
Name: amount_spent, dtype: category
Categories (2, object): ['Preferred Customers' < 'VIP']

In [5]:
sum_byInvoice['customer_status'] = pd.qcut(sum_byInvoice['amount_spent'],2, labels=amount_spentLabels,precision=0)
sum_byInvoice.sample(10)

,amount_spent,customer_status
CustomerID,,
17107,10815.43,VIP
12538,363.95,Preferred Customers
13989,1266.08,VIP
15659,306.45,Preferred Customers
15840,355.56,Preferred Customers
16133,14589.04,VIP
13012,1661.84,VIP
16094,1078.23,VIP
14470,461.19,Preferred Customers


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [15]:
# I had to merge the dataframes first, it seems I was working with the wrong one. But I believee the goal was fulfilled
orders_sum_df = pd.merge(orders_df, sum_byInvoice,left_on='CustomerID', right_index=True)

In [70]:
# United Kingdom has more VIP customers from all countries.

orders_sum_df.groupby('Country')['customer_status'].value_counts().sort_values(ascending=False)

Country          customer_status    
United Kingdom   VIP                    299045
                 Preferred Customers     55300
Germany          VIP                      8275
France           VIP                      7413
EIRE             VIP                      7238
                                         ...  
Greece           Preferred Customers        22
Channel Islands  Preferred Customers        17
Bahrain          Preferred Customers        17
Canada           Preferred Customers        16
Saudi Arabia     Preferred Customers         9
Name: customer_status, Length: 61, dtype: int64

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [75]:
# The country wich has the most VIP + Preferred Customers is United Kingdom.

orders_sum_df.groupby("Country").agg({"customer_status": "count"}).sort_values(by="customer_status", ascending=False)

,customer_status
Country,
United Kingdom,354345
Germany,9042
France,8342
EIRE,7238
Spain,2485
Netherlands,2363
Belgium,2031
Switzerland,1842
Portugal,1462
